In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from skimage.io import imread
from skimage.transform import resize
from sklearn.utils import shuffle


In [2]:

# Paths to zip files
train_zip = '/kaggle/input/dogs-vs-cats/train.zip'
test_zip = '/kaggle/input/dogs-vs-cats/test1.zip'
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

# Extract zip files
with zipfile.ZipFile(train_zip, 'r') as z:
    z.extractall('/kaggle/working/')
with zipfile.ZipFile(test_zip, 'r') as z:
    z.extractall('/kaggle/working/')


In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Target image size
image_size = (64, 64)

# Function to load and preprocess training images
def load_training_images(train_dir, image_size):
    # Filter for image files only
    valid_extensions = {".jpg", ".jpeg", ".png"}
    
    # List comprehension to load images and labels
    images, labels = zip(*[
        (resize(imread(os.path.join(train_dir, img_name)), image_size, anti_aliasing=True).flatten(), 
         0 if 'cat' in img_name else 1)
        for img_name in os.listdir(train_dir) 
        if any(img_name.lower().endswith(ext) for ext in valid_extensions)
    ])
    
    return np.array(images), np.array(labels)

# Load training data
X, y = load_training_images(train_dir, image_size)

# Shuffle and split into train and validation sets
X, y = shuffle(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
